# Семинар - создание и наполнение таблиц 

## Работа с python и с Metabase

### Что мы делаем: 
###### 1. Мы учимся подключаться к Metabase через его API c помощью библиотеки METABASEPY
###### 2. Мы создадим маленькую локальную БД на базе движка SQLite. Наши права не позволяют создавать таблицы и представления непосредственно в Metabase, но в своей локальной БД мы имеем полное право делать все, что нам захочется без ограничений! :) 
###### 3. Мы попробуем запускать запросы через Metabasepy, отдавать результат в Pandas Dataframe
###### 4. Наконец, мы попробуем сохранить данные из Pandas Dataframe в виде INSERT
###### 

### Шаг 0. Подготовка к работе (наш инструментарий)

### Шаг 0.1 Установка Metabasepy

##### - Нам надо установить библиотеку Metabasepy и подключиться c с ее помощью к БД через Metabase

In [1]:
!pip install metabasepy

### Инициализация подключения metabasepy к нашей БД

In [1]:
from metabasepy import Client, MetabaseTableParser

cli = Client(
                username="demo10@skillfactory.ru", # тот самый, который вам выдали
                password="t9vQJlErQ9WcMi", # и пароль к нему!
                base_url="http://sql.skillfactory.ru:3000"
      )

cli.authenticate()

### Шаг 0.2 Как нам найти нужную БД (Skillfactory)?

##### - Смотрим, какие БД нам доступны: нас интересует  параметр Id БД, он потребуется для подключения 

In [32]:
cli.databases.get()['data'][2]

{'description': None,
 'features': ['full-join',
  'basic-aggregations',
  'nested-field-columns',
  'standard-deviation-aggregations',
  'persist-models',
  'expression-aggregations',
  'percentile-aggregations',
  'foreign-keys',
  'right-join',
  'left-join',
  'native-parameters',
  'nested-queries',
  'expressions',
  'set-timezone',
  'regex',
  'case-sensitivity-string-filter-options',
  'binning',
  'inner-join',
  'advanced-math-expressions'],
 'cache_field_values_schedule': '0 0 18 * * ? *',
 'timezone': 'UTC',
 'auto_run_queries': True,
 'metadata_sync_schedule': '0 59 * * * ? *',
 'name': 'Skillfactory',
 'caveats': None,
 'creator_id': 1,
 'is_full_sync': True,
 'updated_at': '2022-11-21T14:26:15.403',
 'native_permissions': 'write',
 'cache_ttl': None,
 'is_sample': False,
 'id': 2,
 'is_on_demand': False,
 'options': None,
 'engine': 'postgres',
 'initial_sync_status': 'complete',
 'refingerprint': None,
 'created_at': '2022-10-26T17:20:29.316',
 'points_of_interest': No

### Шаг 0.3 Тестовый запрос, чтобы убедиться, что мы подключились к metabase 

##### - Теперь, зная, что наша БД имеет id=2, можем подклюлчаться и давать запросы в БД.

In [3]:
# Таблица SHIPMENT
# Запрос всех полей без условий

shipment_sql =    """
                    SELECT 
                        ship_id, 
                        cust_id,
                        weight,
                        truck_id,
                        driver_id,
                        city_id,
                        ship_date               
                    FROM 
                        shipping.shipment 
                    WHERE 
                        1=1

                 ;"""

# заготовка шапки для DATAFRAME!
shipment_header = ['ship_id','cust_id','weight', 'truck_id', 'driver_id','city_id','ship_date']
books_header = ['book_name', 'author','language_code','author_rating', 'book_average_rating', 'book_ratings_count', 'genre',
                'publisher','book_id']

query_response = cli.dataset.post(database_id=2, query=shipment_sql)

data_table = MetabaseTableParser.get_table(metabase_response=query_response)

data_table.rows[0:5]


[[1000, 3660, 3528.0, 1, 23, 137, '2016-01-08T00:00:00Z'],
 [1001, 2001, 11394.0, 2, 23, 186, '2016-01-18T00:00:00Z'],
 [1002, 1669, 8712.0, 3, 27, 268, '2016-01-19T00:00:00Z'],
 [1003, 989, 17154.0, 4, 23, 365, '2016-01-24T00:00:00Z'],
 [1004, 2298, 9279.0, 5, 27, 253, '2016-01-26T00:00:00Z']]

### Из-за подлого характера Metabasepy мы сами должны позаботиться о том, чтобы колонки отображались с шапкой. Для этого у нас был заготовлен в Шаге 0 (часть 2) массив с именами колонок. И сейчас мы превратим все в красиво отображающийся DataFrame. 
### P.S. Названия колонок можно найти в Шаге 0 (часть 2) для всех таблиц БД shipment

In [4]:
import pandas as pd
shipment = pd.DataFrame(data_table.rows)
shipment.columns = shipment_header
shipment[0:5]

,ship_id,cust_id,weight,truck_id,driver_id,city_id,ship_date
0,1000,3660,3528.0,1,23,137,2016-01-08T00:00:00Z
1,1001,2001,11394.0,2,23,186,2016-01-18T00:00:00Z
2,1002,1669,8712.0,3,27,268,2016-01-19T00:00:00Z
3,1003,989,17154.0,4,23,365,2016-01-24T00:00:00Z
4,1004,2298,9279.0,5,27,253,2016-01-26T00:00:00Z


### Шаг 0.4. Инициализируем SQLite3 и создаем файл БД, нашего "подопытного кролика"

In [5]:
import sqlite3
# con = sqlite3.connect("d:/data/tutorial.db") #разумеется, тут вам надо указать свой путь к файлу!
con = sqlite3.connect("tutorial.db") #разумеется, тут вам надо указать свой путь к файлу!



In [7]:
#пример создания таблицы

sqlite_create_table = """create table my_test4 (
                        id int,
                        test_name varchar(1000),
                        test_score decimal(5,2),
                        created_on timestamp                        
                                        );"""
sqlite_create_view = """
                    create view my_view as select * from my_test;
"""

cur = con.cursor()
cur.execute(sqlite_create_table)
res = cur.execute("SELECT name FROM sqlite_master where name like 'my_test%'")
res.fetchall()

[('my_test4',)]

### Шаг 0.5. Вставка данных в таблицы


In [8]:
sqlite_insert = """
                    insert into my_test(test_name. test_score, created_on)
                    values ('my first insert test', 4.1, Today())
"""

### Важный пример: а вот так мы можем из массива строк вносить данные с подстановкой значений в запросе. (Почти) так и работает Airflow, с которым мы познакомимся в следующем модуле.

In [ ]:


# data = [
#    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
#    ("Monty Python's The Meaning of Life", 1983, 7.5),
#    ("Monty Python's Life of Brian", 1979, 8.0),
# ]
# cur.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
# con.commit()  # Remember to commit the transaction after executing INSERT.

### А теперь наши задания (шаги 1-3).

In [ ]:
# 1. Мы хотим перенести целиком таблицу books из Metabase в нашу тестовую базу данных sqlite. Повторять типы данных 
#    НЕ обязательно, но постарайтесь сохранить хотя бы, чтобы дата была датой, а цифра - цифрой (касается и ID).
#    Решением будет показ запрос к таблице и показ ее содержимого с названием столбцов через PANDAS

In [40]:
with open('query_result_2023-02-25T16_15_25.031777Z.csv', encoding='utf-8') as file:
    header = file.readline()

In [41]:
books_sql = f'SELECT {header.strip()} FROM public.books WHERE 1=1;'

In [38]:
# books_sql =    """
#                     SELECT 
#                         publishing_year,
#                         book_name,
#                         author,
#                         language_code,
#                         author_rating,
#                         book_average_rating,
#                         book_ratings_count,
#                         genre,
#                         publisher,
#                         book_id
#                     FROM 
#                         public.books 
#                     WHERE 
#                         1=1

#                  ;"""

In [42]:
books_header = header.strip().split(',')

query_response = cli.dataset.post(database_id=2, query=books_sql)

data_table = MetabaseTableParser.get_table(metabase_response=query_response)

In [43]:
data_table.rows[0:2]

[[1975,
  'Beowulf',
  'Unknown, Seamus Heaney',
  'en-US',
  'Novice',
  3.42,
  155903,
  'genre fiction',
  'HarperCollins Publishers',
  1],
 [1987,
  'Batman: Year One',
  "Frank Miller, David Mazzucchelli, Richmond Lewis, Dennis O'Neil",
  'eng',
  'Intermediate',
  4.23,
  145267,
  'genre fiction',
  'HarperCollins Publishers',
  2]]

In [55]:
import pandas as pd
books = pd.DataFrame(data_table.rows)
books.columns = books_header
books.head()

,publishing_year,book_name,author,language_code,author_rating,book_average_rating,book_ratings_count,genre,publisher,book_id
0,1975,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903,genre fiction,HarperCollins Publishers,1
1,1987,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,Intermediate,4.23,145267,genre fiction,HarperCollins Publishers,2
2,2015,Go Set a Watchman,Harper Lee,eng,Novice,3.31,138669,genre fiction,"Amazon Digital Services, Inc.",3
3,2008,When You Are Engulfed in Flames,David Sedaris,en-US,Intermediate,4.04,150898,fiction,Hachette Book Group,4
4,2011,Daughter of Smoke & Bone,Laini Taylor,eng,Intermediate,4.04,198283,genre fiction,Penguin Group (USA) LLC,5


In [42]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   publishing_year      2000 non-null   int64  
 1   book_name            1959 non-null   object 
 2   author               2000 non-null   object 
 3   language_code        1906 non-null   object 
 4   author_rating        2000 non-null   object 
 5   book_average_rating  2000 non-null   float64
 6   book_ratings_count   2000 non-null   int64  
 7   genre                2000 non-null   object 
 8   publisher            2000 non-null   object 
 9   book_id              2000 non-null   int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 156.4+ KB


In [34]:
import sqlite3
# con = sqlite3.connect("d:/data/tutorial.db") #разумеется, тут вам надо указать свой путь к файлу!
con = sqlite3.connect("Skillfactory.db")

Создание таблицы и проверка, что таблица books в базе появилась

In [58]:
sqlite_create_table = """create table books (
                                        publishing_year int,
                                        book_name,
                                        author,
                                        language_code,
                                        author_rating,
                                        book_average_rating decimal(5, 2),
                                        book_ratings_count int,
                                        genre,
                                        publisher,
                                        book_id id                        
                                        );"""

In [36]:
cur = con.cursor()
cur.execute(sqlite_create_table) # таблица уже создана
res = cur.execute("SELECT name FROM sqlite_master where name like 'books%'")
res.fetchall()

[('books',)]

In [48]:
data_table.rows[0]

[1975,
 'Beowulf',
 'Unknown, Seamus Heaney',
 'en-US',
 'Novice',
 3.42,
 155903,
 'genre fiction',
 'HarperCollins Publishers',
 1]

In [50]:
cur.executemany("INSERT INTO books VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", data_table.rows)
con.commit()  # Remember to commit the transaction after executing INSERT.

In [52]:
cur.execute("SELECT * FROM books").fetchall()

[(1975,
  'Beowulf',
  'Unknown, Seamus Heaney',
  'en-US',
  'Novice',
  3.42,
  155903,
  'genre fiction',
  'HarperCollins Publishers',
  1),
 (1987,
  'Batman: Year One',
  "Frank Miller, David Mazzucchelli, Richmond Lewis, Dennis O'Neil",
  'eng',
  'Intermediate',
  4.23,
  145267,
  'genre fiction',
  'HarperCollins Publishers',
  2),
 (2015,
  'Go Set a Watchman',
  'Harper Lee',
  'eng',
  'Novice',
  3.31,
  138669,
  'genre fiction',
  'Amazon Digital Services,  Inc.',
  3),
 (2008,
  'When You Are Engulfed in Flames',
  'David Sedaris',
  'en-US',
  'Intermediate',
  4.04,
  150898,
  'fiction',
  'Hachette Book Group',
  4),
 (2011,
  'Daughter of Smoke & Bone',
  'Laini Taylor',
  'eng',
  'Intermediate',
  4.04,
  198283,
  'genre fiction',
  'Penguin Group (USA) LLC',
  5),
 (2015,
  'Red Queen',
  'Victoria Aveyard',
  'eng',
  'Intermediate',
  4.08,
  83354,
  'genre fiction',
  'Amazon Digital Services,  Inc.',
  6),
 (2011,
  'The Power of Habit',
  'Charles Duhigg

In [ ]:
# 2. Мы хотим на нашей локальной БД создать view с запросом, который будет работать с нашей новой локальной таблицей books 
# и будет выдавать:
# - имя автора, 
# - количество его изданных книг,
# - год издания его последней книги
# - число жанров, в которых автор пишет (или писал)
# - количество издательств, публиковавших автора

# - решением будет считаться показ результата SELECT из вашего VIEW.

Создание представления

In [54]:
sqlit_create_view = '''
                    create view my_view as
                    SELECT
                        author,
                        count(book_id) as CountBooks,
                        max(publishing_year) as MaxPublishingYear,
                        count(distinct genre) as CountGenres,
                        count(DISTINCT publisher) as CountPublisher
                    FROM
                        books
                    GROUP by
                        author
                    ORDER by author
                    '''

cur.execute(sqlit_create_view)

In [ ]:
# sqlite_names_columns = '''
#                         select *
#                         from con.columns
#                         where tablename=n'books'
# '''